In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import cftime
import yaml
import xarray as xr
import rpy2.robjects as ro

#My functions
sys.path.insert(0,'../functions/')
import functions_HeatWavesCities as fun_HWC


In [ ]:
#Read main path
with open('../path_main.txt', 'r') as file:   path_main = file.read()
    
dir_data      = f'{path_main}Data/CMIP6/Variables/'
dir_scripts   = f'{path_main}Scripts/'
dir_names     = f'{path_main}Scripts/Model_lists/'
dir_functions = f'{path_main}Scripts/functions/'
dir_HSI_out   = f'{path_main}Data/CMIP6/HSIs/'
dir_HWMId_out = f'{path_main}Data/CMIP6/HWMId/'
if not os.path.exists(dir_HSI_out):   os.mkdir(dir_HSI_out)
if not os.path.exists(dir_HWMId_out): os.mkdir(dir_HWMId_out)


## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Istanbul', 'Moscow', 'London', 'SaintPetersburg', 'Berlin', 'Madrid', 'Kyiv', 'Rome', 'Paris',
          'Bucharest', 'Minsk', 'Vienna', 'Hamburg', 'Warsaw', 'Budapest', 'Barcelona', 'Munich', 'Kharkiv',
          'Milan', 'Belgrade', 'Prague', 'NizhnyNovgorod', 'Kazan', 'Sofia', 'Brussels', 'Stockholm', 'Oslo',
          'Dublin', 'Lisbon', 'Vilnius', 'Copenhagen', 'Helsinki', 'Athens', 'Amsterdam', 'Riga', 'Zagreb']

#Define HSIs
HSI_names = ['TX']

#Define SSPs
SSPs = ['ssp585']

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)

#Define models and SSPs which should be used
all_models = dict()
all_models['ssp585'] = []
with open(dir_names + 'Models_CMIP6_SSP585.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['ssp585'].append(line[:-1])
        
#Add models for historical
all_models['historical'] = all_models['ssp585']


## Calculate HWMId

In [ ]:
area_sel = 'gridpoint'

#Only performs calculation for gridpoint
if area_sel=='gridpoint':

    #Source R functions
    r = ro.r
    r.source(dir_scripts + 'functions/get_HWMId_vCities.r')

    #Define reference and application years
    app_years = [1981, 2099]
    ref_years_vec = ro.r.c(1981, 2010)
    app_years_vec = ro.r.seq(app_years[0], app_years[1])

    #Loop over RCPs
    for SSP in SSPs:

        #Define scenarios and select models
        scenarios = ['historical', SSP]
        models    = all_models[SSP]
        
        #Loop over models
        for model in models:
            
            print(" -" + model)

            #Loop over HSIs
            for HSI_name in HSI_names:

                #Loop over cities
                for i0, city in enumerate(cities):

                    #Loop over scenarios
                    for i1, scen in enumerate(scenarios):

                        #Get file name
                        dir_HSI = dir_HSI_out + city + '/' + scen + '/'
                        files_HSI = [file for file in os.listdir(dir_HSI) if scen in file and model + '_' in file]
                        files_HSI = [file for file in files_HSI if "-3x3_" not in file]
                        if len(files_HSI)!=1:  sys.exit('File is not unique')

                        #Read data
                        data_read = xr.open_dataset(dir_HSI + files_HSI[0])
                        data_read = data_read[HSI_name].to_dataset(name=HSI_name)
                        
                        #Concatenate historical and future data
                        if i1==0:
                            data_comb = data_read
                        else:
                            data_comb = xr.concat((data_comb, data_read), dim='time')

                    #Collect data for all cities
                    if i0==0:
                        data_coll = data_comb
                    else:
                        data_coll = xr.concat((data_coll, data_comb), dim='city')

                #Create output directory and define file name
                dir_HWMId = dir_HWMId_out + 'HWMId-' + HSI_name + '/'
                if not os.path.exists(dir_HWMId): os.mkdir(dir_HWMId)
                
                #Define file names
                fname_tmp   = dir_HSI + 'tmp_CMIP6' + area_sel + '.nc'
                fname_HWMId = dir_HWMId + 'HWMId-' + HSI_name + "_" + model + '_' + "-".join(scenarios) + "_" + str(app_years[0]) + "-" + str(app_years[1]) + ".nc"

                #Save data temporarily in NetCDF
                data_coll['city'] = cities
                data_coll.to_netcdf(fname_tmp)

                #Calculate HWMId
                r.get_HWMId_vCities(HSI_name, fname_tmp, fname_HWMId, ref_years_vec, app_years_vec, dir_functions)

                #Read data
                with xr.open_dataset(fname_tmp) as ds:
                    data_HSI = ds.load()
                    ds.close()
                with xr.open_dataset(fname_HWMId) as ds:
                    data_HWMId = ds.load()
                    ds.close()

                #Put city names
                attrs = data_HWMId['city'].attrs
                data_HWMId['city'] = cities
                data_HWMId['city'].attrs = attrs

                #Save in file
                if os.path.exists(fname_HWMId): os.remove(fname_HWMId)
                data_HWMId.to_netcdf(fname_HWMId)

                #Remove temporary file
                os.remove(fname_tmp)
                